In [1]:
merged_f = open("../../afesm5/taxonomy/taxdump/merged.dmp")

cur_id2prev_id = {}

while True:
  line = merged_f.readline()

  if not line:
    break

  tokens = line.strip().split("\t|\t")
  cur_id = int(tokens[1].strip('\t|'))
  prev_id = int(tokens[0])

  if cur_id in cur_id2prev_id:
    cur_id2prev_id[cur_id].append(prev_id)
  else:
    cur_id2prev_id[cur_id] = [prev_id]

merged_f.close()

# cur_id2prev_id

In [2]:
f = open("/fast/livinit/esmfold/databases/afesm_names.dmp")
taxName = {}

while True:
    line = f.readline()

    if not line:
        break

    tokens = line.split("\t|\t")
    if tokens[3] != "scientific name\t|\n":
      continue

    taxName[int(tokens[0])] = tokens[1]

f.close()

taxName

{1: 'root',
 2: 'Bacteria',
 6: 'Azorhizobium',
 7: 'Azorhizobium caulinodans',
 9: 'Buchnera aphidicola',
 10: 'Cellvibrio',
 11: 'Cellulomonas gilvus',
 13: 'Dictyoglomus',
 14: 'Dictyoglomus thermophilum',
 16: 'Methylophilus',
 17: 'Methylophilus methylotrophus',
 18: 'Pelobacter',
 19: 'Syntrophotalea carbinolica',
 20: 'Phenylobacterium',
 21: 'Phenylobacterium immobile',
 22: 'Shewanella',
 23: 'Shewanella colwelliana',
 24: 'Shewanella putrefaciens',
 25: 'Shewanella hanedai',
 27: 'halophilic eubacterium NRCC 41227',
 28: 'halophilic eubacterium',
 29: 'Myxococcales',
 31: 'Myxococcaceae',
 32: 'Myxococcus',
 33: 'Myxococcus fulvus',
 34: 'Myxococcus xanthus',
 35: 'Corallococcus macrosporus',
 38: 'Archangium disciforme',
 39: 'Archangiaceae',
 40: 'Stigmatella',
 41: 'Stigmatella aurantiaca',
 42: 'Cystobacter',
 43: 'Cystobacter fuscus',
 44: 'Melittangium',
 45: 'Melittangium lichenicola',
 47: 'Archangium',
 48: 'Archangium gephyra',
 49: 'Polyangiaceae',
 50: 'Chondromyc

In [3]:
f = open("/fast/livinit/esmfold/databases/afesm_nodes.dmp")

kid_parent = {}
taxRank = {}

while True:
    line = f.readline()

    if not line:
        break

    tokens = line.split("\t|\t")

    kid_parent[int(tokens[0])] = int(tokens[1])
    taxRank[int(tokens[0])] = tokens[2]

f.close()

In [4]:
dp = {}

def dfs(cur):
    if not cur in kid_parent or cur == kid_parent[cur]:
        dp[cur] = False
        return dp[cur]
    if cur in dp:
        return dp[cur]

    if taxRank[cur] == 'species':
        dp[cur] = cur
        return cur
    
    next = kid_parent[cur]


    dp[cur] = dfs(next)
    return dp[cur]

for key in kid_parent.keys():
    dfs(key)

In [5]:
N=0
for key, val in dp.items():
  if N == 10:
    break
  N+=1
  print(key, val)

1 False
131567 False
2 False
1224 False
28211 False
356 False
335928 False
6 False
7 7
9 9


In [6]:
f = open(f"../coreness/grouping_w_merged_dmp_gtdb-taxId_taxName_speciesId_speciesName.tsv", "w")

N = 0
for key, val in dp.items():
    if val:
        f.write(f"{key}\t{taxName[key]}\t{val}\t{taxName[val]}\n")
        if key in cur_id2prev_id:
            for prev_id in cur_id2prev_id[key]:
                f.write(f"{prev_id}\t{taxName[key]}\t{val}\t{taxName[val]}\n")

f.close()